## **Problem 10 + 11 + 12**

## Huggingface Trainer Model

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
#import datasets
#from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification
import torch
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
from tqdm import tqdm
import os
import evaluate
from sklearn.dummy import DummyClassifier
#from datasets import load_dataset
from collections import Counter, defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

### Import Data source files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datapath = "drive/My Drive/SI 630/HW3/datasets/"
train_df = pd.read_csv(datapath + "si630w23-hw3-train.csv")
test_df = pd.read_csv(datapath + "si630w23-hw3-test.public.csv")
eval_df = pd.read_csv(datapath + "si630w23-hw3-dev.csv")

In [ ]:
train_df.head()

,Premise,Premise Justification,Premise_ID,Argument,Argument_ID,persuasiveness
0,We don’t actually “know” anything...,When it comes to what we consider something th...,g7243k,Descartes went all through this 400 years ago....,foglc3a,1.00
1,There is no biblical basis for a full ban on a...,Delta update: When you combine a significant n...,etoyxd,Another one to note is that Numbers 5:11 throu...,ffhyp7q,2.80
2,Sports fans are just as lazy and unhealthy as ...,"To preface this post, there's nothing wrong wi...",nw01pe,Are you under the impression that pubic opinio...,h167r53,1.50
3,Pirating media isn't as bad as people make it ...,"I'm not saying that it isn't wrong, but it's n...",hiya60,The music industry has been pretty devastated ...,fwkdiz5,3.25
4,the current way social issues are addressed is...,For context: I am a black man raised in a pred...,cjxhhs,&gt; women are hostile to all men because most...,evh6xw7,4.50


### Data Preprocessing

In [ ]:
train_df['text'] = train_df['Premise Justification'] + '-' +train_df['Argument']
eval_df['text'] = eval_df['Premise Justification'] + '-' +eval_df['Argument']
test_df['text'] = test_df['Premise Justification'] + '-' +test_df['Argument']
train_df.head(3)

,Premise,Premise Justification,Premise_ID,Argument,Argument_ID,persuasiveness,text
0,We don’t actually “know” anything...,When it comes to what we consider something th...,g7243k,Descartes went all through this 400 years ago....,foglc3a,1.0,When it comes to what we consider something th...
1,There is no biblical basis for a full ban on a...,Delta update: When you combine a significant n...,etoyxd,Another one to note is that Numbers 5:11 throu...,ffhyp7q,2.8,Delta update: When you combine a significant n...
2,Sports fans are just as lazy and unhealthy as ...,"To preface this post, there's nothing wrong wi...",nw01pe,Are you under the impression that pubic opinio...,h167r53,1.5,"To preface this post, there's nothing wrong wi..."


In [ ]:
# X_train, y_train = train_df['text'].values, train_df['persuasiveness'].values
# X_val, y_val = eval_df['text'].values, eval_df['persuasiveness'].values
# X_test = test_df['text'].values

# Fancy Huggingface classifiers

We'll use [MiniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased) which is a compressed version of a BERT-like model that is faster to train

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Verify there's at least one GPU sitting around (or not if you want to try CPU!)
torch.cuda.device_count()

1

### Prepare the text

In [ ]:
# Tokenize train and evaluation datasets with BertTokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(data):
    return tokenizer(data, padding=True, truncation=True, max_length=512)

tokenized_X_train = preprocess_function(list(train_df['text']))
tokenized_X_eval = preprocess_function(list(eval_df['text']))

In [ ]:
#tokenized_X_train



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Define a Class to prepare the dataset in desired format to feed in Trainer class
class DataProcess(torch.utils.data.Dataset):
  def __init__(self, encodings, labels = None):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [ ]:
train_dataset = DataProcess(tokenized_X_train, list(train_df['persuasiveness']))
eval_dataset = DataProcess(tokenized_X_eval, list(eval_df['persuasiveness']))

## Train the model

In [ ]:
device = torch.device("cuda")

In [ ]:
from sklearn.metrics import mean_squared_error

def compute_metrics(pred):
  scores = pred.label_ids
  prediction = pred.predictions.squeeze()
  return {"rmse": np.sqrt(mean_squared_error(scores, prediction))}

# Huggingface will generate model checkpoints and these can 
# take up a lot of space, so be sure to set this directory somewhere where
# you can check

# This code block is used so we can have a deterministic training procedure
# (i.e., get the same result each time)
# model_name = "microsoft/MiniLM-L12-H384-uncased"
output_dir = '.'

def model_init():
    from transformers.models.bert.modeling_bert import BertForSequenceClassification
    bertmodel = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1) # label of argument persuasiveness score
    bertmodel = bertmodel.to(device)

    return bertmodel #AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
     
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",
    eval_steps = 500,
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    #weight_decay=0.01,
    do_eval=True,
    seed=1234,
    #save_strategy='epoch',
    #evaluation_strategy='epoch',
    logging_dir=output_dir + 'logs/',
    load_best_model_at_end=True,
    metric_for_best_model="rmse",
    #max_steps = 1000
)

In [ ]:
model_init()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#device = torch.device("cpu")

In [ ]:
trainer = Trainer(
    model_init=model_init,    
    args=training_args,
    train_dataset = train_dataset, #pd.DataFrame(X_train_sparse), #{'input_ids': data_dict["X_train"].toarray(), 'labels': y_train}, #data_dict["X_train"].todense(),  #(X_train_sparse).todense(), 
    eval_dataset = eval_dataset, #pd.DataFrame(X_val_sparse), #{'input_ids': data_dict["X_val"].toarray(), 'labels': y_val}, # #data_dict["X_val"].todense(), #(X_val_sparse).todense(),    
    compute_metrics = compute_metrics,
    tokenizer=tokenizer,
    #data_collator=data_collator,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Rmse
500,0.812600,0.791970,0.889927
1000,0.358700,0.910245,0.954067


TrainOutput(global_step=1208, training_loss=0.5208729687116004, metrics={'train_runtime': 2010.2169, 'train_samples_per_second': 9.611, 'train_steps_per_second': 0.601, 'total_flos': 5083259948605440.0, 'train_loss': 0.5208729687116004, 'epoch': 8.0})

### Evaluation and Predictions

In [ ]:
trainer.evaluate()

{'eval_loss': 0.910244882106781,
 'eval_rmse': 0.9540674686431885,
 'eval_runtime': 18.5979,
 'eval_samples_per_second': 27.799,
 'eval_steps_per_second': 1.774,
 'epoch': 8.0}

In [ ]:
eval_logits = trainer.predict(eval_dataset)
print(eval_logits.predictions.shape, eval_logits.label_ids.shape)

(517, 1) (517,)


In [ ]:
#eval_logits.predictions

In [ ]:
# eval_preds = np.argmax(eval_logits.predictions, axis=0)
# precision_recall_fscore_support(eval_dataset, eval_preds, average='binary')

In [ ]:
tokenized_X_test = preprocess_function(list(test_df['text']))
test_dataset = DataProcess(tokenized_X_test) #, list(test_df['persuasiveness']))

In [ ]:
test_logits = trainer.predict(test_dataset)

In [ ]:
print(test_logits.predictions.shape) #, test_logits.label_ids.shape)
# test_preds = np.argmax(test_logits.predictions, axis=-1)
# precision_recall_fscore_support(test_df.label, test_preds, average='binary')

(518, 1)


In [ ]:
#test_df["Argument_ID"]

In [ ]:
predictions_df = pd.DataFrame([])
predictions_df["Argument_ID"] = test_df["Argument_ID"]
predictions_df["persuasiveness"] = pd.DataFrame(test_logits.predictions)

In [ ]:
predictions_df.shape
predictions_df

,Argument_ID,persuasiveness
0,i3xz4vd,2.658004
1,h34r5lw,4.170889
2,gygklw9,3.115077
3,etvbrbo,1.754498
4,etcjf4q,2.436779
...,...,...
513,ezr9s9d,1.716867
514,gwqf4ys,2.805477
515,i4nz2lt,2.731362
516,gclzuoe,3.287332


In [ ]:
predictions_df.to_csv("hw3_group9_predictions.csv")

### Problem 13. 
(5 points) Make a bar plot of the correlation scores of each group for the three evaluation subsets in the format shown in Figure 1. In a paragraph, describe what you see and discuss the relative impact of each team’s annotations. For example, did some group’s annotations cause the model to perform worse? Were some group’s labels closer to the model’s predictions? In your discussion, review the relevant groups’ guidelines and describe whether specific instructions in their guidelines might be the cause of these differences in scores/performance.

In [ ]:
eval_annot_df = pd.read_csv(datapath + "si630w23-hw3-dev.individual-annotations.csv")
float(eval_annot_df)

In [ ]:
comb_df = eval_annot_df.merge(predictions_df, on='Argument_ID', how='left')
comb_df.persuasiveness_y

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1944   NaN
1945   NaN
1946   NaN
1947   NaN
1948   NaN
Name: persuasiveness_y, Length: 1949, dtype: float32

#### Ground Truth ratings

(1) averaging the scores annotators in g_i

(2) averaging the scores by all other annotators

In [ ]:
def ground_truth(df, groupid):
  gt = df.loc[(df["Group_ID"] == groupid)]
  gt_rest = df.loc[(df["Group_ID"] != groupid)]
  # avg_p = np.mean(gt["persuasiveness"])
  # avg_prest = np.mean(gt_rest["persuasiveness"])
  # avg_c = np.mean(gt["changeability"])
  # avg_crest = np.mean(gt_rest["changeability"])
  # avg_a = np.mean(gt["premise_agreement"])
  # avg_arest = np.mean(gt_rest["premise_agreement"])
  
  return gt, gt_rest #, avg_p, avg_prest, avg_c, avg_crest, avg_a, avg_arest


In [ ]:
# gt1 = eval_annot_df.loc[(eval_annot_df["Group_ID"] == 1)]
# gt1_rest = eval_annot_df.loc[(eval_annot_df["Group_ID"] != 1)]
# #gt1_rest
# avg1 = np.mean(gt1["persuasiveness"])
# avg1_rest = np.mean(gt1_rest["persuasiveness"])
# avg1, avg1_rest

In [ ]:
dataname = ["gt", "gt_rest", "avg_p", "avg_prest", "avg_c", "avg_crest", "avg_a", "avg_arest"]

In [ ]:
gt_dict = {name: [] for name in dataname}

In [ ]:
for id in set(eval_annot_df["Group_ID"]):

  gt1, gt1_rest, avg_p1, avg_prest1, avg_c1, avg_crest1, avg_a1, avg_arest1 = ground_truth(eval_annot_df, id)

  gt_dict["gt"].append(gt1)
  gt_dict["gt_rest"].append(gt1_rest)
  # gt_dict["avg_p"].append(avg_p1)
  # gt_dict["avg_prest"].append(avg_prest1)
  # gt_dict["avg_c"].append(avg_c1)
  # gt_dict["avg_crest"].append(avg_crest1)
  # gt_dict["avg_a"].append(avg_a1)
  # gt_dict["avg_arest"].append(avg_arest1)

In [ ]:
pivot_df = gt1.pivot(index='Argument_ID', columns='Group_ID', values='persuasiveness')

corr_matrix = pivot_df.corr()

# Print the correlation matrix
print(corr_matrix)

ValueError: ignored

### Problem 14. 
(5 points) How does an annotator’s changeability in their views relate to the model’s performance? If an annotator has a highly changeable view of the premise, would that influence their rating in a way that makes it disagree (or agree) with the model? Using the dev data, make a plot for that shows for each annotation in the dataset the annotator’s changeability on the x-axis and the difference between the annotator’s rating and the model’s rating on the y-axis. You can show this as a scatter plot, a line plot, or an lmplot (the last is probably best). In a few sentences, describe what you see and what implications you think the interpretation of this figure has for the model’s performance and/or training.